In [1]:
import mysql.connector

# database setup and structure

This python script will guide you through setting up a database and all tables for the project. The database structure is shown below:

<img src="EngineerTrackPro.jpg">



# Database login
Enter the credatinals to acces a database. The user needs to have all permissions to create a new project in a database
- host: If database is located on the same machine
- user: the user name, usually admin
- password: secrete to login

In [2]:
host="localhost"
user="admin"
password="helloLayka"

mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password
)
mycursor = mydb.cursor()

# Project database creation
A database will be created with the same name as the project. Do not use special characters such as ( ) - & @ * $ | % ~ except for underscore (_). Also do not use SQL or PL/SQL reserved words 

In [3]:
projectName = input("Enter new project name: ")
try:
    mycursor.execute(f"CREATE DATABASE {projectName}")
except Exception as e:
    # ANSI escape codes for red text and bold
    red_bold = "\033[1;31m"
    reset = "\033[0m"
    print(f"{red_bold}Failed to create database: {e}{reset}")

Enter new project name:  test


# Create project tables

In [4]:
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password,
  database=projectName
)

mycursor = mydb.cursor()

In [5]:
mycursor.execute("CREATE TABLE REVIEW        (id INT AUTO_INCREMENT PRIMARY KEY, name  VARCHAR(64),  initials VARCHAR(4))")


## OWNER TABLE

In [6]:
mycursor.execute("CREATE TABLE OWNER         (id INT AUTO_INCREMENT PRIMARY KEY, name  VARCHAR(64),  initials VARCHAR(4),   role VARCHAR(32),   email VARCHAR(256))")

sql = "INSERT INTO OWNER (name, initials, role) VALUES (%s, %s, %s)"
val = [
  ('None', 'None', 'None'),
  ('Bart Beesem',  'BaBe', 'Project leader'),
  ('Flip Kanarie', 'FlKa', 'Mechanical Engineer'),
]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "owner(s) were inserted.")

3 owner(s) were inserted.


## DOC_STATE TABLE

In [7]:
mycursor.execute("CREATE TABLE DOC_STATE     (id INT AUTO_INCREMENT PRIMARY KEY, state VARCHAR(16))")

sql = "INSERT INTO DOC_STATE (state) VALUES (%s)"
val = [
  ('Draft',),
  ('Proposed',),
  ('Qualified',),
]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "owner(s) were inserted.")

3 owner(s) were inserted.


## REVIEW_STATE TABLE

In [8]:
mycursor.execute("CREATE TABLE REVIEW_STATE  (id INT AUTO_INCREMENT PRIMARY KEY, state VARCHAR(16))")

sql = "INSERT INTO REVIEW_STATE (state) VALUES (%s)"
val = [
  ('Unassigned',),
  ('Assigned',),
  ('Fixed',),
]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "owner(s) were inserted.")

3 owner(s) were inserted.


# CATEGORY TABLE

In [9]:
mycursor.execute("CREATE TABLE CATEGORY      (id INT AUTO_INCREMENT PRIMARY KEY, name  VARCHAR(64),  rank int(4))")

sql = "INSERT INTO CATEGORY (name, rank) VALUES (%s,%s)"
val = [
    ('Concept Specifications','1'), 
    ('Module Specifications','2'), 
    ('Detailed Design Specification','3'),
    ('Test Requirement Specification','4'),
    ('Test Specifications','5')
]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "Categorie(s) were inserted.")


5 Categorie(s) were inserted.


# Documents

In [10]:
s = (
    "id INT AUTO_INCREMENT PRIMARY KEY",
    "name VARCHAR(64)",
    "initials VARCHAR(4)",
    "owner_id int",
    "category_id int",
    "FOREIGN KEY (owner_id) REFERENCES OWNER(id)",
    "FOREIGN KEY (category_id) REFERENCES CATEGORY(id)"
)

# Join the tuple into a single string with commas
s = ", ".join(s)

# Execute the SQL command to create the table
mycursor = mydb.cursor()
mycursor.execute(f"CREATE TABLE DOCUMENT ({s})")

sql = "INSERT INTO DOCUMENT (name, initials, owner_id, category_id) VALUES (%s, %s, %s, %s)"
val = [

    ('Customer Requirement Specifications',  'CRS',  '1', '1'),
    ('Regolatory Requirement Specifications','RRS',  '1', '1'),
    ('Hardware Requirement Specifications',  'HwRS', '1', '2'),
    ('Software Requirement Specifications',  'SwRS', '1', '2'),
    ('Hardware Design Specifications',       'HwDS', '1', '3'),
    ('Software Design Specifications',       'SwDS', '1', '3'),
    ('Hardware Test Specifications',         'HwTS', '1', '4'),
    ('Software Test Specifications',         'SwTS', '1', '4')
    

]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "Document(s) were inserted.")


8 Document(s) were inserted.


# SPEC_TYPE

In [11]:
mycursor.execute("CREATE TABLE SPEC_TYPE (id INT AUTO_INCREMENT PRIMARY KEY, SPEC_TYPE_NAME VARCHAR(255), TYPE ENUM('Function', 'Property', 'Constrain', 'Budget'))")


sql = "INSERT INTO SPEC_TYPE (SPEC_TYPE_NAME, TYPE) VALUES (%s,%s)"
val = [
  ('Has backup-power', 'Function'),
  ('Connector', 'Property'),
  ('Environmental', 'Constrain'),
  ('maximum cost', 'Budget'),
]

mycursor.executemany(sql, val)
mydb.commit()

print(mycursor.rowcount, "Spec_types(s) were inserted.")

4 Spec_types(s) were inserted.


# Specification

In [12]:
s = (
    "id INT AUTO_INCREMENT PRIMARY KEY",
    "title VARCHAR(64)",
    "owner_id INT",
    "document_id INT",
    "spec_parent VARCHAR(256)", 
    "date DATETIME",
    "state_id INT",
    "review INT",
    "spec_type_id INT",
    "FOREIGN KEY (owner_id) REFERENCES OWNER(id)",
    "FOREIGN KEY (document_id) REFERENCES DOCUMENT(id)",
    "FOREIGN KEY (state_id) REFERENCES REVIEW_STATE(id)",
    "FOREIGN KEY (spec_type_id) REFERENCES SPEC_TYPE(id)"
)

# Join the tuple into a single string with commas
s = ", ".join(s)

# Execute the SQL command to create the table
mycursor = mydb.cursor()
mycursor.execute(f"CREATE TABLE SPECIFICATION ({s})")
